<a href="https://colab.research.google.com/github/zwzheng45/Qyin/blob/main/%E9%80%9A%E5%BE%8B%E5%8D%83%E9%9F%B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Please set to use GPU acceleration before running this notebook: Runtime → Change runtime type → Hardware accelerator → T4 GPU    
note: Colab free version only provide old NVIDIA T4 GPU so it will be quite slow when running prediction
***
## 请在运行前设置使用GPU加速：代码执行程序→更改运行时类型→硬件加速器→T4 GPU
注：由于免费版Colab只能使用英伟达的上古T4 GPU，所以推理速度比较缓慢

In [ ]:
# @title Clone and install llama-factory and all its dependencies｜克隆并安装llama-factory以及所需的依赖
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
!pip install -e ".[torch,metrics]" --use-deprecated=legacy-resolver
print("That error up there is FINE. It's because stupid pip can never resolve its dependencies correctly")

Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 315, done.
remote: Counting objects: 100% (315/315), done.
remote: Compressing objects: 100% (245/245), done.
remote: Total 315 (delta 80), reused 157 (delta 57), pack-reused 0 (from 0)
Receiving objects: 100% (315/315), 8.94 MiB | 12.20 MiB/s, done.
Resolving deltas: 100% (80/80), done.
/content/LLaMA-Factory
Obtaining file:///content/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/

In [ ]:
# @title Install flash attention｜安装flash attention
# @markdown Flash attention is used to speed up the prediction process. It's fine if these part failed to install.

# @markdown Flash attention用于加速推理并减少显存占用，这一步安装即使报错了也不影响使用。
!pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 28.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.6.3-cp310-cp310-linux_x86_64.whl size=187309225 sha256=237ef9c6157db394e1ddde4ba609a21ebb98382377a27041edc09318801a6f24
  Stored in directory: /root/.cache/pip/wheels/7e/e3/c3/89c7a2f3c4adc07cd1c675f8bb7b9ad4d18f64a72bccdfe826
Successfully built flash-attn


In [ ]:
# @title Install deepspeed (for multi GPU only)｜安装deepspeed（仅限多GPU环境）
!pip install deepspeed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.1 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.15.1-py3-none-any.whl size=1483860 sha256=509a8688e5c871b78f9508c99a5e654029ff179aae42a8081f44636a7d23c9e1
  Stored in directory: /root/.cache/pip/wheels/da/cb/14/9cbba50c73df044eb32a7ca29e34844c5f8959e12d22ae8b60
Successfully built deepspeed


In [ ]:
# @title Download pre-trained model｜下载预训练模型
# @markdown This could take up a long time to finish (around 10 minutes)

# @markdown 可能需要10分钟左右的时间来完成下载

%cd /content/LLaMA-Factory
!mkdir models
%cd models

!git lfs install

!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/ZZW45/Qyin
%cd Qyin
!GIT_LFS_SKIP_SMUDGE=1 git lfs pull

/content/LLaMA-Factory
/content/LLaMA-Factory/models
Updated git hooks.
Git LFS initialized.
Cloning into 'Qyin'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 34 (delta 5), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (34/34), 3.60 MiB | 3.77 MiB/s, done.
/content/LLaMA-Factory/models/Qyin


In [ ]:
# @title Download cleaned dataset｜下载已清理数据集（仅训练时需要）
%cd /content/LLaMA-Factory/data
!wget https://musicllm.demo.rayann.cn/file/abc_dataset.json

/content/LLaMA-Factory/data
--2024-09-18 19:57:36--  https://musicllm.demo.rayann.cn/file/abc_dataset.json
Resolving musicllm.demo.rayann.cn (musicllm.demo.rayann.cn)... 104.21.60.19, 172.67.187.184, 2606:4700:3037::6815:3c13, ...
Connecting to musicllm.demo.rayann.cn (musicllm.demo.rayann.cn)|104.21.60.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/json]
Saving to: ‘abc_dataset.json’

abc_dataset.json        [    <=>             ]  14.55M  18.9MB/s    in 0.8s    

2024-09-18 19:57:38 (18.9 MB/s) - ‘abc_dataset.json’ saved [15253641]



In [ ]:
# @title Add dataset info (for training only)｜添加数据集信息（仅训练时需要）
%cd /content/LLaMA-Factory/data

import json

with open('dataset_info.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

data["abc_dataset"] = {
    "file_name": "abc_dataset.json",
    "columns": {
        "prompt": "instruction",
        "query": "input",
        "response": "output"
    }
}

with open('dataset_info.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=2)

/content/LLaMA-Factory/data


In [ ]:
# @title Start llama-factory｜LLama Factory，启动！！
# @markdown # Click run and wait, when appears, click the link after `Running on public URL:` to enter webui.
# @markdown # 点击运行后等待，出现`Running on public URL:`时点击后面的链接进入webui
# @markdown ***
# @markdown # For inference
# @markdown 1. Switch to the tab **Chat**
# @markdown 2. Set the **Model name** as ```Qwen2-7B-Chat``` and **Model path** as ```./models/Qyin```
# @markdown 3. Click **Load model**
# @markdown 4. When the model is loaded successfully, scroll down and change **Maximum new tokens** to `4096`
# @markdown 5. Input prompt and click **Submit**

# @markdown # 进行推理
# @markdown 1. 左上角**lang**切换到**zh**以切换到中文界面
# @markdown 2. 进入**Chat**选项卡
# @markdown 3. 将上面的**模型名称**改为```Qwen2-7B-Chat```，**模型路径**改为```./models/Qyin```
# @markdown 4. 点击**加载模型**
# @markdown 5. 模型加载完成后，向下滑动页面找到**最大生成长度**并将值改为`4096`
# @markdown 6. 在输入框中输入提示并按**提交**开始生成

%cd /content/LLaMA-Factory
!GRADIO_SHARE=true llamafactory-cli webui

/content/LLaMA-Factory
2024-09-18 20:17:08.163256: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-18 20:17:08.196071: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-18 20:17:08.205628: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-18 20:17:08.238092: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-18 20:17:10.614569: W 

# Below code is to transfer ABC notation that was generated by model into MIDI files and sound file
# 以下代码用于将模型输出的ABC记谱法转换为MIDI文件以及音频

In [ ]:
# @title Download and compile abcmidi｜下载并编译abcmidi
%cd /content
!wget https://master.dl.sourceforge.net/project/abcmidi/abcMIDI-2024.08.13.zip
!unzip abcMIDI-2024.08.13.zip
!rm abcMIDI-2024.08.13.zip
%cd abcmidi
!make
!chmod +x abc2midi
!sudo apt install -y fluidsynth
!pip install --upgrade pyfluidsynth
!pip install pretty_midi

/content
--2024-09-18 20:12:26--  https://master.dl.sourceforge.net/project/abcmidi/abcMIDI-2024.08.13.zip
Resolving master.dl.sourceforge.net (master.dl.sourceforge.net)... 216.105.38.12
Connecting to master.dl.sourceforge.net (master.dl.sourceforge.net)|216.105.38.12|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.sourceforge.net/project/abcmidi/abcMIDI-2024.08.13.zip [following]
--2024-09-18 20:12:27--  https://downloads.sourceforge.net/project/abcmidi/abcMIDI-2024.08.13.zip
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Connecting to downloads.sourceforge.net (downloads.sourceforge.net)|204.68.111.105|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://master.dl.sourceforge.net/project/abcmidi/abcMIDI-2024.08.13.zip?viasf=1 [following]
--2024-09-18 20:12:27--  https://master.dl.sourceforge.net/project/abcmidi/abcMIDI-2024.08.13.zip?viasf=1
Connecting to

In [ ]:
# @title Input ABC notation text here｜在此输入ABC记谱法文本
# @markdown ### Please paste the abc notation text to `raw_abc_data` **keeping the line breaks the the triple quotation marks**.

# @markdown ### 将待转换的abc记谱法文本粘贴进`raw_abc_data`中，保留abc乐谱的换行以及前后的三重双引号

# @markdown ***

# @markdown #### It shall looks like this:

# @markdown #### 例如：

# @markdown <code>raw_abc_data = """<br>X: 1<br>T: happy, melodic, energetic, love, film<br>M: 2/2<br>L: 1/8<br>Q:1/4=120<br>K:F % 1 flats<br>V:1<br>%%clef treble<br>%%MIDI program 0<br>G,,D, G,[DA,] [GB,-][AB,-] [BB,-][d-B,]| \ <br>....<br>"""</code>

%cd /content/abcmidi
import os





# ↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
# modify this part ↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↙↙↙↙↙↙↙↙↙↙↙↙↙↙↙↙↙↙↙↙↙↙↙↙↙↙↙
raw_abc_data = """
(put the abc notation text here, replacing this text)
"""
# modify this part ↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↖↖↖↖↖↖↖↖↖↖↖↖↖↖↖↖↖↖↖↖↖↖↖↖↖↖↖
# ↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑







with open('temp.abc', 'w') as file:
    file.write(raw_abc_data)

!./abc2midi temp.abc

import pretty_midi
from IPython import display

pm = pretty_midi.PrettyMIDI('temp1.mid')

print("\nPlease wait for the result to be converted\n↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓")

def display_audio(pm: pretty_midi.PrettyMIDI):
  _SAMPLING_RATE = 44100
  waveform = pm.fluidsynth(fs=_SAMPLING_RATE)
  return display.Audio(waveform, rate=_SAMPLING_RATE)

display_audio(pm)